<a href="https://colab.research.google.com/github/danzi13/WeeklyMileageDistribution/blob/main/Strava-MileageMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install stravaio folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 154 kB 5.9 MB/s 
     |████████████████████████████████| 58 kB 1.8 MB/s 
     |████████████████████████████████| 155 kB 22.9 MB/s 
     |████████████████████████████████| 288 kB 40.0 MB/s 
     |████████████████████████████████| 749 kB 39.1 MB/s 
     |████████████████████████████████| 489 kB 42.5 MB/s 
  Created wheel for snaptime: filename=snaptime-0.2.4-py3-none-any.whl size=3593 sha256=fb32befc2aacc16ab1f1bccd8190bb803bd9ad8050088f3e7a9a91ae3f8e0455
  Stored in directory: /root/.cache/pip/wheels/e5/f2/0e/994dee3bb4e289095bfd938b21e542c8fd09e35a4d5fae6b06
Successfully built snaptime
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [ ]:
import os
import logging
import json
import urllib
import requests
import folium
from stravaio import StravaIO
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from datetime import date
import time
import webbrowser

In [ ]:
client_id = '*****'
client_secret = '**********'
refresh_token = '*********'
access_token = '**********'

code = "**********"

In [ ]:
params_oauth = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": f"http://localhost:8000/authorization_successful",
    "scope": "read,profile:read_all,activity:read",
    "state": 'https://github.com/sladkovm/strava-http', # Sladkovm is the author of the Stravaio library
    "approval_prompt": "force"
}
values_url = urllib.parse.urlencode(params_oauth)
base_url = 'https://www.strava.com/oauth/authorize'
authorize_url = base_url + '?' + values_url
print(authorize_url)

https://www.strava.com/oauth/authorize?client_id=87593&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A8000%2Fauthorization_successful&scope=read%2Cprofile%3Aread_all%2Cactivity%3Aread&state=https%3A%2F%2Fgithub.com%2Fsladkovm%2Fstrava-http&approval_prompt=force


In [ ]:
AUTHORIZATION_CODE = "**************"

In [ ]:
payload = {
    "client_id": client_id,
    "client_secret": client_secret,
    "grant_type": "authorization_code",
    "code": AUTHORIZATION_CODE,
}

response = requests.request(
    "POST", "https://www.strava.com/api/v3/oauth/token", data=payload
)

response = json.loads(response.text)
TOKEN = response["access_token"]

In [ ]:
client = StravaIO(access_token=TOKEN)
athlete = client.get_logged_in_athlete()
activities = client.get_logged_in_athlete_activities(after='last year')

Fetched 30, the latests is on 2021-07-29 19:16:21+00:00
Fetched 30, the latests is on 2021-08-22 13:35:13+00:00
Fetched 30, the latests is on 2021-09-20 15:51:13+00:00
Fetched 30, the latests is on 2021-10-18 13:56:00+00:00
Fetched 30, the latests is on 2021-11-24 17:35:49+00:00
Fetched 30, the latests is on 2021-12-18 20:32:42+00:00
Fetched 30, the latests is on 2022-01-16 16:17:00+00:00
Fetched 30, the latests is on 2022-02-10 16:52:43+00:00
Fetched 30, the latests is on 2022-03-09 15:02:16+00:00
Fetched 30, the latests is on 2022-04-06 01:15:23+00:00
Fetched 30, the latests is on 2022-05-02 15:58:11+00:00
Fetched 30, the latests is on 2022-05-26 23:32:00+00:00
Fetched 30, the latests is on 2022-06-29 14:00:11+00:00
Fetched 8, the latests is on 2022-07-06 01:11:48+00:00


In [ ]:
import pandas as pd

In [ ]:
column_names = ["distance", "weekday","start_date"]

df = pd.DataFrame(columns = column_names)

for i in range(len(activities)):
  if(activities[i].type == 'Run'):
    df2 = {'distance': activities[i].distance, 'weekday': activities[i].start_date.weekday(),'start_date':activities[i].start_date}
    df = df.append(df2, ignore_index = True)
  


In [ ]:
start = df[df.weekday==0].first_valid_index()

#might have to switch due to long run ending on whatever day 
end = df[df.weekday==5].last_valid_index() + 1

In [ ]:
df = df.iloc[start:end]

In [ ]:
df = df.reset_index()

In [ ]:
del df['index']

In [ ]:
column_names = ["Mon","m2","Tues","tu2","Wed","w2","Thurs","th2","Fri","f2","Sat","sa2","Sun","su2"]

ml_df = pd.DataFrame(columns = column_names)

In [ ]:
j = 0
ml_df.loc[j, ml_df.iloc[:,2 * df.loc[0].weekday].name] = df.loc[0].distance / 1609

In [ ]:
j = 0 #current row value / doesn't entirely work but whatever 
for i in range(len(df)-1): #always adding / analyzing value i+1

  #if value < prev_value
  if(df.loc[i+1].weekday < df.loc[i].weekday):
    j = j + 1 #new row & add the value into new row
    ml_df.loc[j, ml_df.iloc[:,2 * df.loc[i+1].weekday].name] = df.loc[i+1].distance / 1609
    continue

  # value > 7 day = new row / this shouldn't impact my data specfically (never had a week fully off)
  if(df.loc[i+1].start_date.day - df.loc[i].start_date.day > 6 and df.loc[i+1].start_date.month == df.loc[i].start_date.month):
    j = j + 1 
    ml_df.loc[j, ml_df.iloc[:,2 * df.loc[i+1].weekday].name] = df.loc[i+1].distance / 1609
    continue

  if(df.loc[i+1].weekday == df.loc[i].weekday):
    if(df.loc[i+1].start_date.hour - df.loc[i].start_date.hour > 6): #6 hour difference = double 
      ml_df.loc[j, ml_df.iloc[:,2 * df.loc[i+1].weekday + 1].name] = df.loc[i+1].distance / 1609

    else: 
      ml_df.loc[j, ml_df.iloc[:,2 * df.loc[i+1].weekday].name] = df.loc[i].distance / 1609 + df.loc[i+1].distance / 1609
  
  else: 
    ml_df.loc[j, ml_df.iloc[:,2 * df.loc[i+1].weekday].name] = df.loc[i+1].distance / 1609


ml_df = ml_df.fillna(0)
ml_df = ml_df.sort_index()

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [ ]:
tot_dist = ml_df.sum(axis = 1)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(tot_dist.to_numpy().reshape(-1,1), ml_df, test_size = 0.3, random_state=0)
data = LinearRegression()
#x_train = tot_dist.to_numpy().reshape(-1,1)
data.fit(x_train, y_train)
y_pred = data.predict(x_test)
print(y_pred)

[[ 8.88979864  0.35185149  9.26639046  0.52247334  9.21476453  1.42063984
   7.99710969  0.83549795  9.27495308  0.3473985   4.49598801  0.71782546
  12.02330936  0.36630047]
 [ 9.3116303   0.40445147  9.69809736  0.59463069  9.92892794  1.5460691
   8.6072627   0.94672289 10.00001105  0.34859759  4.66138166  0.80721638
  12.96317527  0.36175103]
 [ 8.54443578  0.30878672  8.91294251  0.46339653  8.63006322  1.31794814
   7.49756398  0.74443564  8.68133215  0.34641679  4.36057659  0.64463914
  11.25382045  0.3700252 ]
 [ 8.83676592  0.34523861  9.21211622  0.51340171  9.12497983  1.40487086
   7.92040119  0.82151474  9.18379871  0.34724775  4.47519471  0.70658722
  11.90514932  0.36687243]
 [ 8.62717694  0.31910407  8.99762068  0.47755     8.77014448  1.34255076
   7.61724388  0.76625212  8.82355035  0.34665198  4.39301812  0.66217293
  11.43817266  0.36913284]
 [ 9.2991797   0.40289895  9.68535529  0.59250093  9.90784901  1.54236698
   8.5892537   0.94344002  9.97861056  0.3485622   4

In [ ]:
#whatever total mileage you need to run for the week
data.predict([[75]])

array([[ 9.76796588,  0.46135389, 10.16511594,  0.67269019, 10.70150669,
         1.6817579 ,  9.26732348,  1.06704552, 10.7843755 ,  0.34989475,
         4.84030376,  0.90391907, 13.97991797,  0.35682946]])

For the most part quite accurate, aside from wanting to double everyday, I have found that you could combine the miles from the doubles to one / two days. But it does show Sunday Long Run (longest run of the week), Saturday as low mileage easy day, Wednesday as 2nd longest run of the week. And monday/tues/thurs as around the same distance and used for getting some miles in. 